In [7]:
import boto3

In [8]:
# Initialize boto3 handler
rds = boto3.client('rds')

# Describes your currently active databases
DBs = rds.describe_db_instances()['DBInstances']
activeDBs = [ x['DBName'] for x in DBs ]
activeDBs

['finalproject']

In [9]:
DB_NAME = 'finalproject'
INSTANCE_NAME = 'final-instance'

db = rds.describe_db_instances()['DBInstances'][0]
ENDPOINT = db['Endpoint']['Address']
PORT = db['Endpoint']['Port']
DBID = db['DBInstanceIdentifier']

print(DBID,
      "is available at", ENDPOINT,
      "on Port", PORT,
     )

final-instance is available at final-instance.cjasscu8073h.us-east-1.rds.amazonaws.com on Port 3306


In [17]:
db_url = 'mysql+mysqlconnector://{}:{}@{}:{}/{}'.format('username',
                                                              'password',
                                                              ENDPOINT,
                                                              PORT,
                                                              DB_NAME)

In [12]:
import mysql.connector

# Connect to the MySQL database
conn =  mysql.connector.connect(host=ENDPOINT, 
                                user="username", 
                                passwd="password", 
                                port=PORT, 
                                database=DB_NAME)
cur = conn.cursor()

In [13]:
cur.execute("SELECT COUNT(*) FROM simulation_data")
print(cur.fetchall())

[(4000,)]


In [16]:
conn.close()

In [14]:
import dask
from dask.distributed import Client

In [27]:
client = Client()
client

/Users/yingrongmao/anaconda3/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 51562 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:51562/status,
Dashboard: http://127.0.0.1:51562/status,Workers: 5
Total threads: 10,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51563,Workers: 5
Dashboard: http://127.0.0.1:51562/status,Total threads: 10
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:51580,Total threads: 2
Dashboard: http://127.0.0.1:51582/status,Memory: 3.20 GiB
Nanny: tcp://127.0.0.1:51566,


In [56]:
import dask.dataframe as dd
from sqlalchemy import create_engine
import pandas as pd

# Create SQL Alchemy engine
engine = create_engine(db_url)

# Define the table and columns you want to load
table_name = 'simulation_data'
# columns = ["combination_id", "num_nodes", "num_new_edges", "time_lst"]  # specify columns you want to read

# Read data from SQL into a Dask DataFrame
ddf = dd.read_sql_table(table_name, con=db_url, index_col='RunId', npartitions=5)


print(ddf.head(3))


                         combination_id  RunId  iteration  Step  num_tasks  \
0  num_tasks8num_nodes10num_new_edges1s      0          0   100          8   
1  num_tasks8num_nodes10num_new_edges2s      1          0   100          8   
2  num_tasks8num_nodes10num_new_edges3s      2          0   100          8   

   num_nodes  num_new_edges  skills_proportion  prob_memory  availability  \
0         10              1                0.1          0.5           0.5   
1         10              2                0.1          0.5           0.5   
2         10              3                0.1          0.5           0.5   

                                            time_lst  \
0  [89, 50, 48, 44, 46, 50, 47, 44, 57, 48, 38, 4...   
1  [72, 53, 43, 52, 37, 39, 44, 52, 43, 52, 44, 3...   
2  [93, 79, 64, 46, 43, 48, 41, 50, 41, 39, 42, 3...   

                                  actor_sequence_lst  
0  [{0: 3, 1: 1, 2: 8, 3: 5, 4: 7, 5: 4, 6: 6, 7:...  
1  [{0: 2, 1: 2, 2: 0, 3: 0, 4: 0, 5: 7, 6: